d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Reading and writing from CosmosDB

## Learning Objectives
By the end of this lesson, you should be able to:
* Write data into Cosmos DB
* Read data from Cosmos DB

Cosmos DB requires some dependencies; details on configuration can be found [here](https://docs.databricks.com/data/data-sources/azure/cosmosdb-connector.html).

##![Spark Logo Tiny](https://files.training.databricks.com/images/wiki-book/general/logo_spark_tiny.png) Load Cosmos DB

Now load a small amount of data into Cosmos to demonstrate that connection

In [0]:
%run ./Includes/Classroom-Setup

<span>1.</span> Enter the CosmosDB connection information into the cell below. <br>

In [0]:
PrimaryKey = dbutils.secrets.get(scope="demo", key="cosmoskey")
URI = dbutils.secrets.get(scope="demo", key="cosmosuri")
CosmosDatabase = "dbretail"
CosmosCollection = "rating"

cosmosConfig = {
  "Endpoint": URI,
  "Masterkey": PrimaryKey,
  "Database": CosmosDatabase,
  "Collection": CosmosCollection
}

<span>2.</span> Read the input parquet file.

In [0]:
from pyspark.sql.functions import col
ratingsDF = (spark.read
  .parquet("dbfs:/mnt/training/initech/ratings/ratings.parquet/")
  .withColumn("rating", col("rating").cast("double")))
print("Num Rows: {}".format(ratingsDF.count()))

In [0]:
display(ratingsDF)

<span>3.</span> Write the data to Cosmos DB.

In [0]:
ratingsSampleDF = ratingsDF.sample(.0001)

(ratingsSampleDF.write
  .mode("overwrite")
  .format("com.microsoft.azure.cosmosdb.spark")
  .options(**cosmosConfig)
  .save())

<span>4.</span> Confirm that your data is now in Cosmos DB.

In [0]:
dfCosmos = (spark.read
  .format("com.microsoft.azure.cosmosdb.spark")
  .options(**cosmosConfig)
  .load())

display(dfCosmos)


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>